In [ ]:
!pip install torch==2.8.0
!pip install torchaudio==2.8.0
!pip install torchvision==0.23.0
!pip install flash-attn --no-build-isolation #--no-cache-dir
!pip install --upgrade transformers
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 139.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from transformers import GPT2Config
from transformers import GPT2LMHeadModel

In [ ]:
import os
import glob
import orjson
import signal
import torch
from tqdm import tqdm
import wandb
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adafactor
# from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from transformers import (
    AutoModelForCausalLM,
    PreTrainedTokenizerFast,
    GPT2Config,
)
from scipy.stats import spearmanr
import pandas as pd
import torch.nn as nn
from torch.utils.data import random_split

class STSDataset(Dataset):
  def __init__(self, file_path, tokenizer, max_len=128):
    self.tokenizer = tokenizer
    self.max_len = max_len

    self.data = pd.read_csv(file_path, sep='\t', quoting=3, dtype=str, keep_default_na=False).fillna('') #quoting=3은 따옴표 무시

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    row = self.data.iloc[idx]
    sentence1 = str(row.sentence1)
    sentence2= str(row.sentence2)
    score = float(row.score)

    try:
        inputs1 = self.tokenizer(sentence1, return_tensors='pt', max_length=self.max_len, padding='max_length', truncation=True)
    except Exception as e:
        print(f"[STS ERROR] sentence1 idx={idx}: {repr(sentence1)}")
        print("Exception:", e)
        # 건너뛰려면 리턴 None 혹은 임의값, 혹은 exception 재발생
        return None

    try:
        inputs2 = self.tokenizer(sentence2, return_tensors='pt', max_length=self.max_len, padding='max_length', truncation=True)
    except Exception as e:
        print(f"[STS ERROR] sentence2 idx={idx}: {repr(sentence2)}")
        print("Exception:", e)
        return None

    return {
        'input_ids1': inputs1['input_ids'].squeeze(0),
        'attention_mask1': inputs1['attention_mask'].squeeze(0),
        'input_ids2': inputs2['input_ids'].squeeze(0),
        'attention_mask2': inputs2['attention_mask'].squeeze(0),
        'score': torch.tensor(score, dtype=torch.float)
    }

class STSModel(nn.Module):
  def __init__(self, pretrained_model):
    super().__init__()
    self.model = pretrained_model
    self.config = self.model.config
    self.regression_head = nn.Sequential(
        nn.Linear(self.config.hidden_size * 3, self.config.hidden_size),
        nn.ReLU(),
        nn.Linear(self.config.hidden_size, 1)
    )

 # 토큰 임베딩 평균
  def _mean_pooling(self, model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# 문장1, 문장2 임베딩, 차이 벡터 합쳐 *MLP 회귀
  def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
    outputs1 = self.model(input_ids=input_ids1, attention_mask=attention_mask1)
    outputs2 = self.model(input_ids=input_ids2, attention_mask=attention_mask2)
    embedding1 = self._mean_pooling(outputs1, attention_mask1)
    embedding2 = self._mean_pooling(outputs2, attention_mask2)
    diff = torch.abs(embedding1 - embedding2)
    combined_embedding = torch.cat([embedding1, embedding2, diff], dim=1)
    return self.regression_head(combined_embedding)


# STS 미세튜닝 함수
def findtune_and_evaluate_sts(
    main_model,
    sts_train_dataloader,
    sts_dev_dataloader,
    device,
    finetune_epochs=3,
    lr=2e-5
):
  print("STS 파인튜닝 시작")

  base_model = main_model.transformer if hasattr(main_model, 'transformer') else main_model
  sts_model = STSModel(base_model).to(device)

  optimizer = Adafactor(sts_model.parameters(), lr=lr)
  loss_fn = nn.MSELoss()

  best_spearman = -1.0

  for epoch in range(finetune_epochs):
    sts_model.train()
    total_train_loss = 0
    for batch in sts_train_dataloader:
      optimizer.zero_grad()

      input_ids1 = batch['input_ids1'].to(device)
      attention_mask1 = batch['attention_mask1'].to(device)
      input_ids2 = batch['input_ids2'].to(device)
      attention_mask2 = batch['attention_mask2'].to(device)
      scores = batch['score'].to(device)

      with autocast():
        predicted_scores = sts_model(input_ids1, attention_mask1, input_ids2, attention_mask2).squeeze(-1)
        loss = loss_fn(predicted_scores, scores)

      total_train_loss += loss.item()

      loss.backward()
      optimizer.step()

    avg_train_loss = total_train_loss / len(sts_train_dataloader)
    print(f"STS 파인튜닝 Epoch {epoch+1} - Avg Train Loss: {avg_train_loss:.4f}")

    # 매 에포크 후 dev 셋으로 검증
    sts_model.eval()
    real_scores = []
    model_scores = []

    with torch.no_grad():
      for batch in sts_dev_dataloader:
        input_ids1 = batch['input_ids1'].to(device)
        attention_mask1 = batch['attention_mask1'].to(device)
        input_ids2 = batch['input_ids2'].to(device)
        attention_mask2 = batch['attention_mask2'].to(device)

        with autocast():
          predicted_scores = sts_model(input_ids1, attention_mask1, input_ids2, attention_mask2).squeeze(-1)

        model_scores.extend(predicted_scores.cpu().numpy())
        real_scores.extend(batch['score'].cpu().numpy())

    spearman_corr, _ = spearmanr(real_scores, model_scores)
    print(f"STS 파인튜닝 Epoch {epoch+1} - Spearman Correlation: {spearman_corr:.4f}")

    if spearman_corr > best_spearman:
      best_spearman = spearman_corr

  print(f"STS 파인튜닝 및 검증 종료. 최고 점수: {best_spearman:.4f}")

  return best_spearman

# ----------------------------
# 1) Dataset 정의
# ----------------------------
class JsonlTextDataset(Dataset):
    def __init__(
        self,
        data_dir: str,
        tokenizer: PreTrainedTokenizerFast,
        block_size: int = 2048,
        stride: int = 256,
    ):
        self.examples = []

        bos_id = tokenizer.bos_token_id
        eos_id = tokenizer.eos_token_id
        pad_id = tokenizer.pad_token_id

        files = glob.glob(os.path.join(data_dir, "*.jsonl"))
        for path in files:
            with open(path, "rb") as f:
                for line_no, line in enumerate(tqdm(f, desc=f'Loading {os.path.basename(path)}', leave=True)):
                    line = line.strip()
                    if not line:
                        continue

                    try:
                        obj = orjson.loads(line)
                    except orjson.JSONDecodeError as e:
                        print(f"[JSONL ERROR] {path} line {line_no}: malformed JSON")
                        print("  >>", line)
                        continue

                    text = obj.get("text", "").strip()
                    if not text:
                        continue

                    # 토크나이저 에러 처리
                    try:
                        raw_ids = tokenizer.encode(text, add_special_tokens=False)
                    except Exception as e:
                        print(f"[TOKENIZER ERROR] {path} line {line_no}: cannot tokenize text")
                        print("  >>", repr(text))
                        print("Exception:", e)
                        continue

                    if len(raw_ids) == 0:
                        continue

                    raw_ids = [bos_id] + raw_ids + [eos_id]

                    # 2) 슬라이딩 윈도우
                    step = block_size - stride
                    for start in range(0, len(raw_ids), step):
                        chunk = raw_ids[start : start + block_size]

                        if len(chunk) < block_size:
                            chunk = chunk + [pad_id] * (block_size - len(chunk))

                        self.examples.append(torch.tensor(chunk, dtype=torch.long))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids = self.examples[idx]
        return {
            "input_ids": input_ids,
            "labels":    input_ids.clone()  # causal LM
        }

def find_latest_checkpoint(ckpt_dir: str):
    """
    ckpt_dir/checkpoint-<step>.pt 파일들 중
    가장 step 이 큰 파일 경로, 그리고 그 스텝 번호를 리턴합니다.
    없으면 (None, None).
    """
    paths = glob.glob(os.path.join(ckpt_dir, "checkpoint-*.pt"))
    if not paths:
        return None, None
    # 파일명에서 숫자만 추출 (checkpoint-1234.pt → 1234)
    def extract_step(path):
        base = os.path.basename(path)
        num = base.replace("checkpoint-", "").replace(".pt", "")
        return int(num)
    steps = [extract_step(p) for p in paths]
    idx = int(steps.index(max(steps)))
    return paths[idx], steps[idx]

def save_checkpoint(output_dir, model, optimizer, scheduler, step):
    os.makedirs(output_dir, exist_ok=True)
    tmp_path   = os.path.join(output_dir, f"checkpoint-{step}.pt.tmp")
    final_path = os.path.join(output_dir, f"checkpoint-{step}.pt")

    # 1) temp 파일에 저장
    with open(tmp_path, "wb") as f:
        torch.save({
            "step":      step,
            "model":     model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
        }, f)
        f.flush()
        os.fsync(f.fileno())   # 디스크에 완전 기록 보장

    # 2) atomic rename
    os.replace(tmp_path, final_path)
    print(f"*** Saved checkpoint: {final_path}")

def register_signal_handlers(output_dir, model, optimizer, scheduler, get_step_fn):
    def handler(signum, frame):
        step = get_step_fn()
        print(f"\n=== Received signal {signum}, saving final checkpoint ...")
        save_checkpoint(output_dir, model, optimizer, scheduler, step)
        exit(0)

    signal.signal(signal.SIGINT, handler)   # Ctrl-C
    signal.signal(signal.SIGTERM, handler)  # kill

def get_model_info(model):
   # ============================================================================
    # 1. 기본 모델 정보
    # ============================================================================
    print("\n [모델 기본 정보]")
    print(f"모델 타입: {model.config.model_type}")
    print(f"모델 아키텍처: {model.__class__.__name__}")

    # ============================================================================
    # 2. 모델 설정(config) 확인
    # ============================================================================
    print("\n [모델 설정 정보]")
    print(f"히든 레이어 크기 (hidden_size): {model.config.hidden_size}")
    print(f"디코더 레이어 수 (num_hidden_layers): {model.config.num_hidden_layers}")
    print(f"어텐션 헤드 수 (num_attention_heads): {model.config.num_attention_heads}")

    # Phi-3는 intermediate_size가 없을 수 있으므로 안전하게 처리
    if hasattr(model.config, 'intermediate_size'):
        print(f"중간 레이어 크기 (intermediate_size): {model.config.intermediate_size}")
    else:
        # FFN 크기는 보통 hidden_size의 4배
        estimated_intermediate = model.config.hidden_size * 4
        print(f"중간 레이어 크기 (추정): {estimated_intermediate}")

    print(f"어휘 크기 (vocab_size): {model.config.vocab_size}")
    print(f"최대 시퀀스(문맥) 길이 (max_position_embeddings): {model.config.max_position_embeddings}")


    # 추가 정보 (Phi-3 특화)
    if hasattr(model.config, 'num_key_value_heads'):
        print(f"Key-Value 헤드 수 (GQA): {model.config.num_key_value_heads}")
    if hasattr(model.config, 'rope_theta'):
        print(f"RoPE Theta: {model.config.rope_theta}")
    if hasattr(model.config, 'sliding_window'):
        print(f"Sliding Window: {model.config.sliding_window}")

    # ============================================================================
    # 3. 전체 파라미터 통계
    # ============================================================================
    print("\n [파라미터 통계]")

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen_params = total_params - trainable_params

    print(f"전체 파라미터: {total_params:,}개 ({total_params / 1e9:.2f}B)")
    print(f"학습 가능 파라미터: {trainable_params:,}개 ({trainable_params / 1e9:.2f}B)")
    print(f"고정 파라미터: {frozen_params:,}개")

    # 메모리 사용량 추정
    mem_fp32 = (total_params * 4) / (1024 ** 2)
    mem_fp16 = mem_fp32 / 2
    print(f"예상 메모리:")
    print(f"  • Float32: {mem_fp32:.2f} MB ({mem_fp32/1024:.2f} GB)")
    print(f"  • Float16: {mem_fp16:.2f} MB ({mem_fp16/1024:.2f} GB)")

    # ============================================================================
    # 4. 모델 구조 계층 분석
    # ============================================================================
    print("\n [모델 구조 계층]")

    # 모델의 주요 컴포넌트 확인
    print("주요 컴포넌트:")
    for name, module in model.named_children():
        module_params = sum(p.numel() for p in module.parameters())
        print(f"  • {name}: {module_params:,}개 파라미터")


    # ============================================================================
    # 5. 카테고리별 파라미터 분석
    # ============================================================================
    print("\n [카테고리별 파라미터 분석]")

    categories = {
        'Embeddings  (임베딩)': 0,
        'Attention   (어텐션)': 0,
        'MLP/FFN (피드포워드)': 0,
        'LayerNorm   (정규화)': 0,
        'Output Head (출력층)': 0,
        'Others        (기타)': 0
    }

    for name, param in model.named_parameters():
        param_count = param.numel()

        # 카테고리 분류
        if 'embed' in name.lower():
            categories['Embeddings  (임베딩)'] += param_count
        elif 'attn' in name.lower() or 'attention' in name.lower():
            categories['Attention   (어텐션)'] += param_count
        elif 'mlp' in name.lower() or 'ffn' in name.lower() or 'fc' in name.lower():
            categories['MLP/FFN (피드포워드)'] += param_count
        elif 'norm' in name.lower() or 'ln' in name.lower():
            categories['LayerNorm   (정규화)'] += param_count
        elif 'lm_head' in name.lower() or 'output' in name.lower():
            categories['Output Head (출력층)'] += param_count
        else:
            categories['Others        (기타)'] += param_count

    print(f"{'카테고리':<30} {'파라미터 수':<20} {'비율'}")
    print("-" * 65)

    for category, count in categories.items():
        if count > 0:
            percentage = (count / total_params) * 100
            print(f"{category:<20} {count:>20,}개 {percentage:>6.2f}%")

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
cached_dataset_path = "cached_full_dataset.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) 토크나이저 / 데이터셋
tokenizer = PreTrainedTokenizerFast.from_pretrained("coreAI_tokenizer")
block_size = 2048
stride = 256
if os.path.exists(cached_dataset_path):
  print("전처리 데이터셋 파일이 존재하여 불러옵니다.")
  full_dataset = torch.load(cached_dataset_path,weights_only=False)
else:
  print("전처리 데이터셋 파일이 존재하지 않아 새로 생성합니다.")
  full_dataset = JsonlTextDataset("data", tokenizer, block_size=block_size, stride=stride)
  print("전처리 데이터셋 생성 완료")
  torch.save(full_dataset, cached_dataset_path)
  print("데이터셋 저장 완료")

split_rate = 0.975
train_size = int(split_rate*len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))
grad_accum_steps = 32
per_device_batch_size = 512 // grad_accum_steps


전처리 데이터셋 파일이 존재하지 않아 새로 생성합니다.


Loading 한국어_데이터셋_3_1.jsonl: 459857it [27:58, 25.34it/s]

In [ ]:
grad_accum_steps = 64
per_device_batch_size = 512 // grad_accum_steps

In [ ]:
train_dl = DataLoader(
    train_ds,
    batch_size=per_device_batch_size,
    shuffle=True,
    drop_last=False
)

val_dl = DataLoader(
    val_ds,
    batch_size=per_device_batch_size,
    shuffle=False,
    drop_last=False
)

In [ ]:
try:
  sts_dev_dataset = STSDataset("data/STSData/sts-dev.tsv", tokenizer)
  sts_dev_dataloader = DataLoader(sts_dev_dataset, batch_size=per_device_batch_size, shuffle=False)
  sts_train_dataset = STSDataset("data/STSData/sts-train.tsv", tokenizer)
  sts_train_dataloader = DataLoader(sts_train_dataset, batch_size=per_device_batch_size, shuffle=True)
  run_sts_eval = True
  print("STS 데이터셋 로드 완료")
except FileNotFoundError:
  print("Warning: STS 데이터셋이 없습니다. STS 평가는 건너뜁니다.")
  run_sts_eval = False

In [ ]:
bos_id = tokenizer.bos_token_id
eos_id = tokenizer.eos_token_id
pad_id = tokenizer.pad_token_id
unk_id = tokenizer.unk_token_id

# 2) 모델 & optimizer & scheduler & scaler
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_embd=768,
    n_layer=12,
    n_head=12,
    n_positions=block_size,
    bos_token_id=bos_id,
    eos_token_id=eos_id,
    pad_token_id=pad_id,
    unk_token_id=unk_id,
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1,
)
model = AutoModelForCausalLM.from_config(config,attn_implementation="flash_attention_2",).to(device)
model.resize_token_embeddings(len(tokenizer))
optimizer = Adafactor(model.parameters(), lr=3e-4, weight_decay=0.01)
total_steps = 50000
max_grad_norm = 1.0
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=total_steps, eta_min=1e-6
)

scaler = GradScaler()

# 3) 체크포인트 복원 시도
ckpt_dir = "checkpoints"
latest_ckpt, last_step = find_latest_checkpoint(ckpt_dir)
if latest_ckpt:
    print(f"Loading checkpoint {latest_ckpt} (step={last_step}) …")
    ckpt = torch.load(latest_ckpt, map_location=device)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    scheduler.load_state_dict(ckpt["scheduler"])
    global_step = ckpt["step"]
else:
    print("No checkpoint found, starting from scratch.")
    global_step = 0

wandb.login(key="d5b25fa78b19fef961f3f6b203f821bd5d2c5b91")
wandb.init(
    project="CoreAI_pretrained",           # W&B 웹에서 만들 프로젝트 이름
    name=f"run-{os.getpid()}",            # (옵션) 실험 이름
    config={                              # hyperparameter 로깅
        "lr":           3e-4,
        "batch_size":   per_device_batch_size,
        "grad_accum":   grad_accum_steps,
        "total_steps":  total_steps,
        "max_grad_norm": max_grad_norm,
        "model_config": config.to_dict(), # 모델 configuration 도 함께
    }
)

print(get_model_info(model))

# 모델 파라미터/그래디언트 추적(logging) (옵션)
wandb.watch(model, log="all", log_freq=100)

# 4) 학습 루프
validation_interval = 200
model.train()
register_signal_handlers(ckpt_dir, model, optimizer, scheduler, get_step_fn=lambda: global_step)

for epoch in range(1):
    pbar = tqdm(train_dl, desc=f"Epoch {epoch}", leave=True)
    for step, batch in enumerate(pbar):
        input_ids = batch["input_ids"].to(device)
        labels    = batch["labels"].to(device)

        with autocast():
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss / grad_accum_steps

        scaler.scale(loss).backward()

        if (step + 1) % grad_accum_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

            global_step += 1

            if global_step % validation_interval == 0:
                model.eval()
                total_val_loss = 0.0
                num_val_batches = 0
                with torch.no_grad():
                    print("검증 중")
                    for val_batch in val_dl:
                        val_input_ids = val_batch["input_ids"].to(device)
                        val_labels    = val_batch["labels"].to(device)

                        with autocast():
                            val_outputs = model(input_ids=val_input_ids, labels=val_labels)
                            total_val_loss += val_outputs.loss.item()
                            num_val_batches += 1

                avg_val_loss = total_val_loss / num_val_batches
                perplexity = torch.exp(torch.tensor(avg_val_loss)).item()
                print(f"*** Validation Step {global_step} - Loss: {avg_val_loss:.4f}, Perplexity: {perplexity:.2f}")
                wandb.log({
                    "eval/loss":      avg_val_loss,
                    "eval/perplexity": perplexity,
                }, step=global_step)

                if run_sts_eval:
                  spearman_correlation = findtune_and_evaluate_sts(model, sts_train_dataloader, sts_dev_dataloader, device)
                  wandb.log({
                      "sts/spearman": spearman_correlation
                  }, step=global_step)

                model.train()


            cur_loss = loss.item() * grad_accum_steps
            cur_lr   = scheduler.get_last_lr()[0]

            # tqdm
            pbar.set_postfix({
                "step": global_step,
                "loss": f"{cur_loss:.4f}",
                "lr":   f"{cur_lr:.2e}"
            })

            # --------------------------------------------
            # (★) W&B 에 로그 기록
            # --------------------------------------------
            wandb.log({
                "train/loss": cur_loss,
                "train/lr":   cur_lr,
                "step":       global_step
            }, step=global_step)

            # if global_step % 100 == 0:
            #     # loss 에 grad_accum_steps 곱해서 원래 스케일로 복원
            #     print(f"Epoch {epoch} Step {global_step} Loss {(loss.item() * grad_accum_steps):.4f}")

            # 매 100 스텝마다 체크포인트 저장
            if global_step > 0 and global_step % 100 == 0:
                save_checkpoint(ckpt_dir, model, optimizer, scheduler, global_step)

            if global_step >= total_steps:
                break

    if global_step >= total_steps:
        break

if global_step > 0:
    save_checkpoint(ckpt_dir, model, optimizer, scheduler, global_step)

# 2-4) 모델&토크나이저 저장
model_dir = "foundation_ckpt"
os.makedirs(model_dir, exist_ok=True)
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# 아티팩트 생성 및 로깅
artifact = wandb.Artifact(
    name="gpt2-pretrained",   # 아티팩트 이름
    type="model",             # 유형(모델, 데이터셋 등)
    metadata={"step": global_step}
)
artifact.add_dir(model_dir)
wandb.log_artifact(artifact)

# 실험 종료 알림
wandb.finish()